<a href="https://colab.research.google.com/github/halen48/SocketsDemo/blob/main/Cliente_Servidor_(Single_Cell).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import socket
from multiprocessing import Process
from threading import Thread, Lock
import time
import random
import pickle

In [ ]:
velocidade = 1.5 #@param {type:"slider", min:0.1, max:2, step:0.1}

<a href="https://docs.python.org/dev/library/socket.html#socket.socket.listen"> Documentação de Socket</a>

In [ ]:
class Server():
  def __init__(self, end, cod, max_clients, tamanho_buffer):
    self.endereco = end
    self.codificacao = cod
    self.max_clients = max_clients
    self.lista_clientes = {}
    self.mutex = Lock()
    self.tamanho_buffer = tamanho_buffer
    self.ultimo_id = 0
    self.clientes_conectados = []

  def adicionar_cliente(self, conn, addr):

    id = self.ultimo_id
    self.ultimo_id += 1
    thread = Thread(target = self.parse_cliente, args=(conn, addr, id))
    self.lista_clientes[id] = ( (conn, thread) )
    self.lista_clientes[id][1].start()

    print("[Servidor] %d/%d clientes conectados"%(len(self.lista_clientes), self.max_clients))

    return id
  
  def checar_conexoes_clientes(self):
    while True:
      #limpa os clientes que estão desconectados
      self.clientes_conectados = list(self.lista_clientes.keys())
      for c_id in self.clientes_conectados:
        if (not self.lista_clientes[c_id][1].is_alive()):
          self.remover_cliente(c_id)
      #Caso a lista tenha espaço para mais clientes, a rotina encerra por aqui
      if (len(self.lista_clientes) < self.max_clients):
          return
      #Se tem mais clientes que o servidor pode suportar, aguarda...
      print("[Servidor] Muitos clientes conectados. Aguardando...")
      print("[Servidor - Debug] IDs Conectados: ", self.clientes_conectados)
      time.sleep(0.5/velocidade)

  def gerenciador_clientes(self):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
      while True:
        try:
          s.bind(self.endereco)
          s.listen(self.max_clients)
          time.sleep(0.5/velocidade)
          break
        except Exception as e:
          print(e)
          print("Tentando vincular conexão...")
          continue
      
      while True:
        
        self.checar_conexoes_clientes()  

        conn, addr = s.accept()
        print('[Servidor] Foi conectado um cliente em:', addr)
        self.adicionar_cliente(conn, addr)
      #s.close()
  
  def remover_cliente(self,index):
    print('[Servidor] Desconectando o cliente[%d]: %s'%(index,self.lista_clientes[index][0]))
    self.lista_clientes[index][0].close()
    del self.lista_clientes[index]
    print("[Servidor] %d/%d clientes conectados"%(len(self.lista_clientes), self.max_clients))
    

  def parse_cliente(self,conn, addr, id):
    while True:
      try:

        data = conn.recv(tamanho_buffer)
        print('\033[3%dm'%(1+id%5)+'[Servidor] recebi', repr(data), 'do cliente <id: %d>'%id,'\x1b[0m')
        if not data:
            break
        
        data = data.decode(self.codificacao)

        valor = random.randint(0,50)

        if(data == 'negativo'):
          valor *= -1
        
        data = pickle.dumps( (id,valor) )
        time.sleep(0.5/velocidade)
        conn.sendall(data)
      except Exception as e:
        print("[Servidor]", e)
        continue
    
  
  def run(self):
    self.gerenciador_clientes()


In [ ]:
class Client():
  def __init__(self, end, cod, tamanho_buffer):
    self.endereco = end
    self.codificacao = cod
    self.tamanho_buffer = tamanho_buffer
    self.id = None

  def connect(self, s):
    while True:
      try:
        s.connect(self.endereco)
        break
      except ConnectionRefusedError:
        #print('[Cliente] Tentando conectar...')
        time.sleep(2)
      except TimeoutError:
        if(self.id):
          print('\033[3%dm'%(1+self.id%5)+'[Cliente] Timeout! <id:%d>'%self.id,'\x1b[0m')
        else:
          print("[Cliente] Timeout! <id: Não conectado>")

  def rotina(self):
  
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
      self.connect(s)
      while True:
        time.sleep(1/velocidade)
        mensagem = random.choice(['positivo','negativo'])
        s.sendall(bytes(mensagem,self.codificacao))
        data = pickle.loads ( s.recv(tamanho_buffer) )

        if( not self.id):
          self.id = data[0]
        
        print('\033[3%dm'%(1+self.id%5)+'[Cliente] recebi', data[1], 'do servidor <id:%d>'%self.id,'\x1b[0m')
        if(random.randint(0,5) == 0):
          print('\033[3%dm'%(1+self.id%5)+'[Cliente] vou desligar... <id:%d>'%self.id,'\x1b[0m')
          break
        #s.close()
  
  def run(self):
    self.rotina()

In [ ]:
endereco = ('127.0.0.1', 33333)
codificacao = 'latin-1'
tamanho_buffer = 2048

s = Server(endereco,codificacao, 5, tamanho_buffer)

p = Process(target=s.run)
p.start()

while True:
  time.sleep(random.randint(1,3)/5/velocidade)
  Thread(target = Client(endereco,codificacao, tamanho_buffer).run ).start()
  
  
p.join()